In [1]:
import numpy as np
import cv2
from fractions import Fraction
from sympy import diff, Symbol
import sympy
import numpy as np
import math
from sympy import *
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist

cam=cv2.VideoCapture(-1)        
# construct the argument parse and parse the arguments
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5, 5))
# load the image
#image = cv2.imread('Long.png')
#image = cv2.resize(image, (1000, 750))
while True:
    ret,image=cam.read()

    # define the list of boundaries
    thres = 40
    boundaries = [
        ([0, 0, 230], [50, 50, 255]),#red
        ([0, 100, 230], [30, 180, 255]),#orange
        ([230, 0, 0], [255, 50,50 ]) # blue    
    ]
    clone = image.copy()
    cv2.imshow('getCamera',clone)
    # loop over the boundaries
    ax=[]
    ay=[]
    for (lower, upper) in boundaries:
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
        # find the colors within the specified boundaries and apply
        # the mas
        mask = cv2.inRange(image, lower, upper)
        output1 = cv2.bitwise_and(image, image, mask = mask)
        x = output1.nonzero()
        ax.append(x[1][0])
        ay.append(x[0][0])
    x_center=[]
    y_center=[]
    for i in ax:
        x_center.append(i-2000)
    for i in ay:
        y_center.append(i-1500)
    print("x_center",x_center)
    print("y_center",y_center)
    f=4.73#mm
    sensor_size=[6.4,4.8]#mm
    X1,Y1,Z1=0,0,0#A
    X2,Y2,Z2=27.5,5.8,0#B cm
    ratio=Distance(X1,Y1,Z1,X2,Y2,Z2)/(Distance(x_center[0],y_center[0],0,x_center[1],y_center[1],0)*1.6*0.0001)#cm
    print("ratio",int(ratio))
    S2real=f*0.1*ratio#cm
    for i in range(5):
        S = symbols('S')
        S=solve((1/S)+(1/S2real)-(1/f),S)[0]
        #print("S",S)
        S2real = symbols('S2real')
        S2real=solve((1/S)+(1/S2real)-(1/f),S2real)[0]
    print("S2real",S2real)#cm   
    print("S",S)#cm
    if 0xFF&cv2.waitKey(5)==27:
        break
cv2.destroyAllWindows()
        

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def Distance_o(x1,y1,S2real):#for calculate the distance of the pixel
    dist = sympy.sqrt((sympy.sqrt((x1**2+y1**2))*ratio*0.1)**2+S**2)
    return dist
X3,Y3,Z3=21,-8.3,0#cm


x1,y1=x_center[0],y_center[0]#pixel
x2,y2=x_center[1],y_center[1]#pixel
x3,y3=x_center[2],y_center[2]#pixel
camera_real_x=((0-x1)*1.6*0.0001*ratio)
camera_real_y=((0-y1)*1.6*0.0001*ratio)
camera_real_z=S2real#initial pose for camera cm

A = Symbol('A',real=True)
B = Symbol('B',real=True)
C = Symbol('C',real=True)
angle1=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2)))
angle2=sympy.acos(np.dot((x1*ratio*0.1-A,y1*ratio*0.1-B,(S+S2real)-C),(x2*ratio*0.1-A,y2*ratio*0.1-B,(S+S2real)-C))/(Distance_o(x1,y1,S)*Distance_o(x2,y2,S)))
angle3=sympy.acos(np.dot((X2-A,Y2-B,Z2-C),(X3-A,Y3-B,Z3-C))/(Distance(A,B,C,X2,Y2,Z2)*Distance(A,B,C,X3,Y3,Z3)))
angle4=sympy.acos(np.dot((x2*ratio*0.1-A,y2*ratio*0.1-B,(S+S2real)-C),(x3*ratio*0.1-A,y3*ratio*0.1-B,(S+S2real)-C))/(Distance_o(x2,y2,S)*Distance_o(x3,y3,S)))

delt_A=diff(angle1-angle2, A)
delt_B=diff(angle1-angle2, B)
delt_C=diff(angle1-angle2, C)

delt_A_2=diff(angle3-angle4, A)
delt_B_2=diff(angle3-angle4, B)
delt_C_2=diff(angle3-angle4, C)

for i in range(5000):
    delt_A=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_A_2=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B_2=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C_2=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    camera_real_x=camera_real_x-2*delt_A-2*delt_A_2
    camera_real_y=camera_real_y-2*delt_B-2*delt_B_2
    camera_real_z=camera_real_z-1*delt_C-1*delt_C_2
    #print(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2))<1)
    angle1=sympy.acos(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)))
    angle2=sympy.acos(np.dot((x1*ratio*0.1-camera_real_x,y1*ratio*0.1-camera_real_y,(S+S2real)-camera_real_z),(x2*ratio*0.1-camera_real_x,y2*ratio*0.1-camera_real_y,(S+S2real)-camera_real_z))/(Distance_o(x1,y1,S)*Distance_o(x2,y2,S)))
    #angle3=sympy.acos(np.dot((X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z),(X3-camera_real_x,Y3-camera_real_y,Z3-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)*Distance(camera_real_x,camera_real_y,camera_real_z,X3,Y3,Z3)))

    print("x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
    print(int(angle1-angle2))
    if Abs(angle1-angle2)<0.5:
        
        print("converge x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
        break
    
        
    
    

In [ ]:
camera_real_x,camera_real_y,camera_real_z=56.211932,15.395214,3.000000
X1,Y1,Z1=100,100,33
X2,Y2,Z2=200,200,33
x1,y1=10,10
x2,y2=20,20
angle1=sympy.acos(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)))
angle2=sympy.acos(np.dot((x1-camera_real_x,y1-camera_real_y,0-camera_real_z),(x2-camera_real_x,x2-camera_real_y,0-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)))
print(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2))<1)

print(angle1-angle2)

In [ ]:
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist
A = Symbol('A')
B = Symbol('B')
C = Symbol('C')

X1,Y1,Z1=100,100,30
X2,Y2,Z2=100,(100+30/sympy.sqrt(3)),30
x1,y1=1,1
x2,y2=2,2


angle1=np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2)
angle2=np.dot((x1-A,y1-B,0-C),(x2-A,x2-B,0-C))/Distance(A,B,C,x1,y1,0)*Distance(A,B,C,x2,y2,0)


a=diff(angle1-angle2, A).subs({A:5,B:10,C:3})
print(a+5)
camera_real_x,camera_real_y,camera_real_z=100,100,0

angle1=np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)
angle2=np.dot((x1-camera_real_x,y1-camera_real_y,0-camera_real_z),(x2-camera_real_x,x2-camera_real_y,0-camera_real_z))/Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)
#print(type(angle1))
print("angle1=%f angle2=%f"%(angle1,angle2))

abs(angle1-angle2)<1
X = Symbol('X',real=True)

diff(sympy.Abs(X**2+2), X).subs({X:5}).n(10)

In [ ]:
X1,Y1,Z1=0,0,30
X2,Y2,Z2=0,(30/sympy.sqrt(3)),0
A,B,C=0,0,0
angle1=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2))
print("angle1=%f"% angle1)
angle1<1
print(angle1.n(10))
sympy.acos(0.2)

In [ ]:
X1,Y1,Z1=0,0,30
X2,Y2,Z2=0,0,0
A,B,C=0,(30/sympy.sqrt(3)),0
angle1=np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2))
print(Distance(A,B,C,X1,Y1,Z1),Distance(A,B,C,X2,Y2,Z2))
print(angle1)
sympy.acos(angle1)